This notebook:
    - Imports an image and its associated mask.
    - Resizes both to 185 x 185.
    - Creates skeletonized masks for the panel images.
    - Performs a stratified 80:20 split of all available images.
        > 80% is available for training.
        > 20% is to be used for validation.
    - Saves them locally.

In [1]:
from skimage.morphology import skeletonize_3d
from skimage.transform import resize
from matplotlib import pyplot as plt
from scipy.misc import imread, imsave
from shutil import rmtree
from glob import glob
from tqdm import tqdm
from os import mkdir
import numpy as np

%matplotlib inline

np.random.seed(42)

In [2]:
IMAGE_SIZE = 185

global_dir  = './Data/170820_Panel_Solos_Sorted_NoDuplicates'

data_dir = { 'cracked_images'    : global_dir + '/Cracked/',
             'cracked_annotated' : global_dir + '/CrackedAnnotated/',
             'uncracked_images'  : global_dir + '/Uncracked/' }

filepaths = { 'cracked_images'    : glob(data_dir['cracked_images']    + "*.png"),
              'cracked_annotated' : glob(data_dir['cracked_annotated'] + "*.png"),
              'uncracked_images'  : glob(data_dir['uncracked_images']  + "*.png") }

### Write cracked panels

In [3]:
counter = 0

# Empty any existing directory contents
for s in ['./Data/170822_Panels/training/images/',
          './Data/170822_Panels/validation/images/',
          './Data/170822_Panels/training/masks/', 
          './Data/170822_Panels/validation/masks/']:
    rmtree(s)
    mkdir(s)

# Write the cracked panels to the folders
for fp in tqdm(filepaths['cracked_images']):
    image_name = fp.split('\\')[-1]
    image      = imread(fp, mode = 'RGB')

    try:
        annotation = imread(data_dir['cracked_annotated'] + image_name)
        mask       = np.logical_and(annotation[:, :, 0] == 255,
                                    annotation[:, :, 1] == 0)

        mask = resize(mask, [185, 185], order = 0)  # Nearest-neighbor interpolation
        mask = skeletonize_3d(mask)
        image = resize(image, [185, 185], order = 0)
        
        # Randomly assign to training/validation folder
        if np.random.rand() > 0.2:
            imsave('./Data/170822_Panels/training/images/' + image_name, image)
            imsave('./Data/170822_Panels/training/masks/' + image_name, mask)
        else:
            imsave('./Data/170822_Panels/validation/images/' + image_name, image)
            imsave('./Data/170822_Panels/validation/masks/' + image_name, mask)
    except:
        counter += 1

  0%|                                                  | 0/911 [00:00<?, ?it/s]C:\Users\admin\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\admin\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
100%|████████████████████████████████████████| 911/911 [00:19<00:00, 47.71it/s]


In [4]:
# Write the uncracked panels to the folders
mask = np.zeros([185, 185], dtype = np.float32)  # Nearest-neighbor interpolation

for fp in tqdm(filepaths['uncracked_images']):
    image_name = fp.split('\\')[-1]
    image      = imread(fp, mode = 'RGB')
        
    image = resize(image, [185, 185], order = 0)

    # Randomly assign to training/validation folder
    if np.random.rand() > 0.2:
        imsave('./Data/170822_Panels/training/images/' + image_name, image)
        imsave('./Data/170822_Panels/training/masks/' + image_name, mask)
    else:
        imsave('./Data/170822_Panels/validation/images/' + image_name, image)
        imsave('./Data/170822_Panels/validation/masks/' + image_name, mask)

  0%|                                                 | 0/5895 [00:00<?, ?it/s]C:\Users\admin\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████████████████████████████████| 5895/5895 [05:14<00:00, 17.92it/s]
